# Plan

``````
Web-3-Wallet-Generator/
├── study/
│   └── wallets
│   └── study.ipynb
├── .gitignore
├── LICENSE
├── README.md
├── pyproject.toml
├── docs/
│   └── index.md
├── tests/
│   └── test_wallet_generator.py
└── web_3_wallet_generator/
    ├── blockchains/
    │   ├── __init__.py
    │   ├── ethereum.py
    │   ├── avalanche.py
    │   └── cosmos.py
    ├── __init__.py
    └── wallet_generator.py
    
``````

In [20]:
!pip install eth_account coincurve pysha3 bech32 ecdsa hdwallets mnemonic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.7 MB/s eta 0:00:00


# Ethereum Wallet

In [5]:
import os
import json
from eth_account import Account

def generate_wallet(blockchain='ethereum', wallet_name=None):
    # Generate a new account
    account = Account.create()

    # Get the private key and address
    wallet = {
        'private_key': account.privateKey.hex(),
        'address': account.address
    }

    # Create the directory if it doesn't exist
    directory = f'wallets/{blockchain}'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # If no wallet name is provided, generate a default one
    if wallet_name is None:
        wallet_number = 1
        wallet_name = f'wallet_{wallet_number}'
        while os.path.exists(f'{directory}/{wallet_name}.json'):
            wallet_number += 1
            wallet_name = f'wallet_{wallet_number}'

    # Save the wallet information to a JSON file
    with open(f'{directory}/{wallet_name}.json', 'w') as file:
        json.dump(wallet, file)

    return wallet

generate_wallet()

{'private_key': '0x4d1d2dac4e508635a379a87e6ca8cb32c303452b80106ca12a475cd7c07c1258',
 'address': '0x111F8ac6341C6832567C0FCbe0b0d09AeD974025'}

# Avalanche

In [10]:
import os
import json
import binascii
from coincurve import PrivateKey
from sha3 import keccak_256

def generate_wallet(blockchain='avalanche', wallet_name=None):
    # Generate a new private key
    private_key = PrivateKey()
    private_key_hex = private_key.to_hex()

    # Get the public key
    public_key = private_key.public_key.format(compressed=False)[1:]

    # Hash the public key and get the address
    address = '0x' + keccak_256(public_key).hexdigest()[-40:]

    # Create the wallet
    wallet = {
        'private_key': private_key_hex,
        'address': address
    }

    # Create the directory if it doesn't exist
    directory = f'wallets/{blockchain}'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # If no wallet name is provided, generate a default one
    if wallet_name is None:
        wallet_number = 1
        wallet_name = f'wallet_{wallet_number}'
        while os.path.exists(f'{directory}/{wallet_name}.json'):
            wallet_number += 1
            wallet_name = f'wallet_{wallet_number}'

    # Save the wallet information to a JSON file
    with open(f'{directory}/{wallet_name}.json', 'w') as file:
        json.dump(wallet, file)

    return wallet

generate_wallet(blockchain='avalanche')

{'private_key': 'ee17c5477c62f55ef90f62d3a1c0af01ed1f7a22d6b9c7c2f5b361dc784940b5',
 'address': '0x6d6f72251dfdb3e8fdf912b8359a77f19da5e113'}

# Cosmos

In [22]:
import os
import json
import hashlib

import bech32
import ecdsa
import hdwallets
import mnemonic


DEFAULT_DERIVATION_PATH = "m/44'/118'/0'/0/0"
DEFAULT_BECH32_HRP = "cosmos"

class BIP32DerivationError(Exception):
    pass

def seed_to_privkey(seed: str, path: str = DEFAULT_DERIVATION_PATH) -> bytes:
    seed_bytes = mnemonic.Mnemonic.to_seed(seed, passphrase="")
    hd_wallet = hdwallets.BIP32.from_seed(seed_bytes)
    derived_privkey = hd_wallet.get_privkey_from_path(path)
    return derived_privkey

def privkey_to_pubkey(privkey: bytes) -> bytes:
    privkey_obj = ecdsa.SigningKey.from_string(privkey, curve=ecdsa.SECP256k1)
    pubkey_obj = privkey_obj.get_verifying_key()
    return pubkey_obj.to_string("compressed")

def pubkey_to_address(pubkey: bytes, *, hrp: str = DEFAULT_BECH32_HRP) -> str:
    s = hashlib.new("sha256", pubkey).digest()
    r = hashlib.new("ripemd160", s).digest()
    five_bit_r = bech32.convertbits(r, 8, 5)
    assert five_bit_r is not None, "Unsuccessful bech32.convertbits call"
    return bech32.bech32_encode(hrp, five_bit_r)

def generate_wallet(blockchain='cosmos', wallet_name=None):
    # Generate a new private key and public key
    while True:
        phrase = mnemonic.Mnemonic(language="english").generate(strength=256)
        try:
            privkey = seed_to_privkey(phrase, path=DEFAULT_DERIVATION_PATH)
            break
        except BIP32DerivationError:
            pass
    pubkey = privkey_to_pubkey(privkey)

    # Hash the public key and get the address
    address = pubkey_to_address(pubkey, hrp=DEFAULT_BECH32_HRP)

    # Create the wallet
    wallet = {
        'private_key': privkey.hex(),
        'address': address
    }

    # Create the directory if it doesn't exist
    directory = f'wallets/{blockchain}'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # If no wallet name is provided, generate a default one
    if wallet_name is None:
        wallet_number = 1
        wallet_name = f'wallet_{wallet_number}'
        while os.path.exists(f'{directory}/{wallet_name}.json'):
            wallet_number += 1
            wallet_name = f'wallet_{wallet_number}'

    # Save the wallet information to a JSON file
    with open(f'{directory}/{wallet_name}.json', 'w') as file:
        json.dump(wallet, file)

    return wallet

generate_wallet(blockchain='cosmos')

{'private_key': '6593318c70af1ebcd96dc86690e808cebc5072894f8fdc20922ee9e6975a31b8',
 'address': 'cosmos1a729vcnn2vu2tc2s2qhcws43yulhk4zgk2ff22'}

# Algorand

In [1]:
!pip install py-algorand-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.9 MB/s eta 0:00:00


In [3]:
from algosdk import account, mnemonic

def create_algorand_wallet():
    private_key, address = account.generate_account()
    print("Your Algorand address: ", address)
    print("Your private key: ", private_key)

    # Convert the private key to a 25-word mnemonic
    mnemonic_phrase = mnemonic.from_private_key(private_key)
    print("Your mnemonic phrase: ", mnemonic_phrase)

create_algorand_wallet()

Your Algorand address:  XTX3KBMPGFB6VNVZTVOQ7LK7QMYG4OS6WDAP2WAQVRXXQKCWWNB6PW2QY4
Your private key:  iRkJQoTDHQ3D8SVkP80F1RnOmG+JCw3Ia+ArQox8OhS877UFjzFD6ra5nV0PrV+DMG46XrDA/VgQrG94KFazQw==
Your mnemonic phrase:  couple caught canal debate describe select mix chalk uncle crunch load outer order tower enter argue accuse quality usual cargo blush palace peace abandon coast


# Bitcoin